In [312]:
import math
from copy import deepcopy

class Vector(object):
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            coordinates=[Decimal(str(x)) for x in coordinates]
            self.coordinates = tuple(coordinates)
            self.dimension = len(coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')


    def __str__(self):
        return 'Vector: {}'.format(self.coordinates)


    def __eq__(self, v):
        return all([abs(x[0]-x[1])<1e-10 for x in zip(self.coordinates,v.coordinates)])
    
    def __add__(self,v):
        return Vector([x[0]+x[1] for x in zip(self.coordinates,v.coordinates)])
        
    def __sub__(self, v):
        return Vector([x[0]-x[1] for x in zip(self.coordinates,v.coordinates)])
        
    def __mul__(self, v):
        return Vector([x*Decimal(str(v)) for x in self.coordinates])
    
    def magnitude(self):
        return Decimal(sum([x**2 for x in self.coordinates])**Decimal('0.5'))
    
    def direction(self):
        if self.magnitude() < 1e-10:
            return 0
        else:
            return Vector([x*(Decimal('1.0')/self.magnitude()) for x in self.coordinates])

    def projection(self,b):
        return b.direction()*dot(self, b.direction()),self + b.direction()*dot(self, b.direction())*-1

def dot(x,y):
    return sum(x[0]*x[1] for x in zip(x.coordinates,y.coordinates))

def angle(x,y):
    return Decimal(math.acos(dot(x,y)/(x.magnitude()*y.magnitude())))

def is_parallel(x,y):
    if all([i<1e-10 for i in x.coordinates]) | all([i<1e-10 for i in y.coordinates]):
        return True
    else:
        return (x.direction() == y.direction()) | (x.direction() == y.direction()*-1)
    
def is_ortogonal(x,y):
    return dot(x,y) < 1e-10

def cross_product(x,y):
    return Vector((x.coordinates[1]*y.coordinates[2]-y.coordinates[1]*x.coordinates[2],\
                   -(x.coordinates[0]*y.coordinates[2]-y.coordinates[0]*x.coordinates[2]),
                   x.coordinates[0]*y.coordinates[1]-y.coordinates[0]*x.coordinates[1]))

from decimal import Decimal, getcontext

getcontext().prec = 30


class Line(object):

    NO_NONZERO_ELTS_FOUND_MSG = 'No nonzero elements found'

    def __init__(self, normal_vector=None, constant_term=None):
        self.dimension = 2

        if not normal_vector:
            all_zeros = ['0']*self.dimension
            normal_vector = Vector(all_zeros)
        self.normal_vector = normal_vector

        if not constant_term:
            constant_term = Decimal('0')
        self.constant_term = Decimal(constant_term)

        self.set_basepoint()


    def set_basepoint(self):
        try:
            n = self.normal_vector
            c = self.constant_term
            basepoint_coords = ['0']*self.dimension

            initial_index = Line.first_nonzero_index(n.coordinates)
            initial_coefficient = n.coordinates[initial_index]

            basepoint_coords[initial_index] = c/initial_coefficient
            self.basepoint = Vector(basepoint_coords)

        except Exception as e:
            if str(e) == Line.NO_NONZERO_ELTS_FOUND_MSG:
                self.basepoint = None
            else:
                raise e


    def __str__(self):

        num_decimal_places = 3

        def write_coefficient(coefficient, is_initial_term=False):
            coefficient = round(coefficient, num_decimal_places)
            if coefficient % 1 == 0:
                coefficient = int(coefficient)

            output = ''

            if coefficient < 0:
                output += '-'
            if coefficient > 0 and not is_initial_term:
                output += '+'

            if not is_initial_term:
                output += ' '

            if abs(coefficient) != 1:
                output += '{}'.format(abs(coefficient))

            return output

        n = self.normal_vector

        try:
            initial_index = Line.first_nonzero_index(n.coordinates)
            terms = [write_coefficient(n.coordinates[i], is_initial_term=(i==initial_index)) + 'x_{}'.format(i+1)
                     for i in range(self.dimension) if round(n.coordinates[i], num_decimal_places) != 0]
            output = ' '.join(terms)

        except Exception as e:
            if str(e) == self.NO_NONZERO_ELTS_FOUND_MSG:
                output = '0'
            else:
                raise e

        constant = round(self.constant_term, num_decimal_places)
        if constant % 1 == 0:
            constant = int(constant)
        output += ' = {}'.format(constant)

        return output
    
    def __eq__(self, v):
        return intersection(self,v) == 'equal'
    
    @staticmethod
    def first_nonzero_index(iterable):
        for k, item in enumerate(iterable):
            if not MyDecimal(item).is_near_zero():
                return k
        raise Exception(Line.NO_NONZERO_ELTS_FOUND_MSG)

class Plane(object):

    NO_NONZERO_ELTS_FOUND_MSG = 'No nonzero elements found'

    def __init__(self, normal_vector=None, constant_term=None):
        self.dimension = 3

        if not normal_vector:
            all_zeros = ['0']*self.dimension
            normal_vector = Vector(all_zeros)
        self.normal_vector = normal_vector

        if not constant_term:
            constant_term = Decimal('0')
        self.constant_term = Decimal(constant_term)

        self.set_basepoint()


    def set_basepoint(self):
        try:
            n = self.normal_vector
            c = self.constant_term
            basepoint_coords = ['0']*self.dimension

            initial_index = Line.first_nonzero_index(n.coordinates)
            initial_coefficient = n.coordinates[initial_index]

            basepoint_coords[initial_index] = c/initial_coefficient
            self.basepoint = Vector(basepoint_coords)

        except Exception as e:
            if str(e) == Line.NO_NONZERO_ELTS_FOUND_MSG:
                self.basepoint = None
            else:
                raise e


    def __str__(self):

        num_decimal_places = 3

        def write_coefficient(coefficient, is_initial_term=False):
            coefficient = round(coefficient, num_decimal_places)
            if coefficient % 1 == 0:
                coefficient = int(coefficient)

            output = ''

            if coefficient < 0:
                output += '-'
            if coefficient > 0 and not is_initial_term:
                output += '+'

            if not is_initial_term:
                output += ' '

            if abs(coefficient) != 1:
                output += '{}'.format(abs(coefficient))

            return output

        n = self.normal_vector

        try:
            initial_index = Line.first_nonzero_index(n.coordinates)
            terms = [write_coefficient(n.coordinates[i], is_initial_term=(i==initial_index)) + 'x_{}'.format(i+1)
                     for i in range(self.dimension) if round(n.coordinates[i], num_decimal_places) != 0]
            output = ' '.join(terms)

        except Exception as e:
            if str(e) == self.NO_NONZERO_ELTS_FOUND_MSG:
                output = '0'
            else:
                raise e

        constant = round(self.constant_term, num_decimal_places)
        if constant % 1 == 0:
            constant = int(constant)
        output += ' = {}'.format(constant)

        return output

    def __eq__(self, v):
        return intersection(self,v) == 'equal'
    
    @staticmethod
    def first_nonzero_index(iterable):
        for k, item in enumerate(iterable):
            if not MyDecimal(item).is_near_zero():
                return k
        raise Exception(Line.NO_NONZERO_ELTS_FOUND_MSG)


class MyDecimal(Decimal):
    def is_near_zero(self, eps=1e-10):
        return abs(self) < eps
    
def intersection(x,y):  
    if is_parallel(x.normal_vector,y.normal_vector):
        if (x.basepoint == y.basepoint):
            return 'equal'
        elif (is_ortogonal(x.basepoint+y.basepoint*(-1),x.normal_vector)):
            return 'equal'
        else:
            return 'parallel'
    else:
        x1 = (y.normal_vector.coordinates[1]*x.constant_term-x.normal_vector.coordinates[1]*y.constant_term)/\
            (x.normal_vector.coordinates[0]*y.normal_vector.coordinates[1]-\
             x.normal_vector.coordinates[1]*y.normal_vector.coordinates[0])
        y1 = (-y.normal_vector.coordinates[0]*x.constant_term+x.normal_vector.coordinates[0]*y.constant_term)/\
            (x.normal_vector.coordinates[0]*y.normal_vector.coordinates[1]-\
             x.normal_vector.coordinates[1]*y.normal_vector.coordinates[0])
        return (x1,y1)

class LinearSystem(object):

    ALL_PLANES_MUST_BE_IN_SAME_DIM_MSG = 'All planes in the system should live in the same dimension'
    NO_SOLUTIONS_MSG = 'No solutions'
    INF_SOLUTIONS_MSG = 'Infinitely many solutions'

    def __init__(self, planes):
        try:
            d = planes[0].dimension
            for p in planes:
                assert p.dimension == d

            self.planes = planes
            self.dimension = d

        except AssertionError:
            raise Exception(self.ALL_PLANES_MUST_BE_IN_SAME_DIM_MSG)


    def swap_rows(self, row1, row2):
        self.planes[row1], self.planes[row2] = self.planes[row2], self.planes[row1]


    def multiply_coefficient_and_row(self, coefficient, row):
        self.planes[row].normal_vector = self.planes[row].normal_vector * Decimal(coefficient)
        self.planes[row].constant_term = self.planes[row].constant_term * Decimal(coefficient)

    def add_multiple_times_row_to_row(self, coefficient, row_to_add, row_to_be_added_to):
        x = deepcopy(self.planes[row_to_add])
        x.normal_vector = x.normal_vector * Decimal(coefficient)
        x.constant_term = x.constant_term * Decimal(coefficient)
        self.planes[row_to_be_added_to].normal_vector = self.planes[row_to_be_added_to].normal_vector + x.normal_vector
        self.planes[row_to_be_added_to].constant_term = self.planes[row_to_be_added_to].constant_term + x.constant_term
        self.planes[row_to_be_added_to].set_basepoint()
        
    def indices_of_first_nonzero_terms_in_each_row(self):
        num_equations = len(self)
        num_variables = self.dimension

        indices = [-1] * num_equations

        for i,p in enumerate(self.planes):
            try:
                indices[i] = p.first_nonzero_index(p.normal_vector.coordinates)
            except Exception as e:
                if str(e) == Plane.NO_NONZERO_ELTS_FOUND_MSG:
                    continue
                else:
                    raise e

        return indices


    def __len__(self):
        return len(self.planes)


    def __getitem__(self, i):
        return self.planes[i]


    def __setitem__(self, i, x):
        try:
            assert x.dimension == self.dimension
            self.planes[i] = x

        except AssertionError:
            raise Exception(self.ALL_PLANES_MUST_BE_IN_SAME_DIM_MSG)


    def __str__(self):
        ret = 'Linear System:\n'
        temp = ['Equation {}: {}'.format(i+1,p) for i,p in enumerate(self.planes)]
        ret += '\n'.join(temp)
        return ret
    
    def compute_triangular_form(self):
        system = deepcopy(self)
        n = len(system.planes)
        for x in range(system.dimension):
            if system.indices_of_first_nonzero_terms_in_each_row()[len(system.planes)-n] != len(system.planes)-n:
                try:
                    system.swap_rows(len(system.planes)-n, system.indices_of_first_nonzero_terms_in_each_row().index(len(system.planes)-n))
                except:
                    continue
            for y in range(x+1,len(system.planes)):
                system.add_multiple_times_row_to_row(-system[y].normal_vector.coordinates[len(system.planes)-n]/system[x].normal_vector.coordinates[len(system.planes)-n],x,y)
            n-=1
            if n == 0:
                break
        return system
    def compute_rref(self):
        tf = self.compute_triangular_form()
        pivot_v = list(reversed(tf.indices_of_first_nonzero_terms_in_each_row()[:tf.indices_of_first_nonzero_terms_in_each_row().index(max(tf.indices_of_first_nonzero_terms_in_each_row()))+1]))
        for i in pivot_v:
            rows = range(len(self.planes))
            piv_c = pivot_v.index(i)
            del rows[piv_c]
            tf.multiply_coefficient_and_row(1/tf.planes[piv_c].normal_vector.coordinates[i],piv_c)
            for j in rows:
                tf.add_multiple_times_row_to_row(-tf.planes[j].normal_vector.coordinates[i]/tf.planes[piv_c].normal_vector.coordinates[i],piv_c,j)
    
        return tf
p0 = Plane(normal_vector=Vector(['1','1','1']), constant_term='1')
p1 = Plane(normal_vector=Vector(['0','1','0']), constant_term='2')
p2 = Plane(normal_vector=Vector(['1','1','-1']), constant_term='3')
p3 = Plane(normal_vector=Vector(['1','0','-2']), constant_term='2')

s = LinearSystem([p0,p1,p2,p3])

print s.indices_of_first_nonzero_terms_in_each_row()
print '{},{},{},{}'.format(s[0],s[1],s[2],s[3])
print len(s)
print s

s[0] = p1
print s

print MyDecimal('1e-9').is_near_zero()
print MyDecimal('1e-11').is_near_zero()

[0, 1, 0, 0]
x_1 + x_2 + x_3 = 1,x_2 = 2,x_1 + x_2 - x_3 = 3,x_1 - 2x_3 = 2
4
Linear System:
Equation 1: x_1 + x_2 + x_3 = 1
Equation 2: x_2 = 2
Equation 3: x_1 + x_2 - x_3 = 3
Equation 4: x_1 - 2x_3 = 2
Linear System:
Equation 1: x_2 = 2
Equation 2: x_2 = 2
Equation 3: x_1 + x_2 - x_3 = 3
Equation 4: x_1 - 2x_3 = 2
False
True


In [292]:
p1 = Plane(normal_vector=Vector(['1','0','0']), constant_term='1')
p2 = Plane(normal_vector=Vector(['2','1','0']), constant_term='2')
s = LinearSystem([p1,p2])
t = s.compute_triangular_form()

In [293]:
print s

Linear System:
Equation 1: x_1 = 1
Equation 2: 2x_1 + x_2 = 2


In [294]:
print t

Linear System:
Equation 1: x_1 = 1
Equation 2: x_2 = 0


In [295]:
print s.compute_rref()

Linear System:
Equation 1: x_1 = 1
Equation 2: x_2 = 0


In [261]:
s.indices_of_first_nonzero_terms_in_each_row()

[0, 0, 0, 0, 0, 0]

In [263]:
t.indices_of_first_nonzero_terms_in_each_row()

[0, 1, 2, -1, -1, -1]

In [264]:
tf=t

In [267]:
tf.indices_of_first_nonzero_terms_in_each_row()[:tf.indices_of_first_nonzero_terms_in_each_row().index(max(tf.indices_of_first_nonzero_terms_in_each_row()))+1]

[0, 1, 2]

In [230]:
print Plane(constant_term='1').basepoint

None


In [231]:
t[1].basepoint == Plane(constant_term='1').basepoint

True

In [286]:
p1 = Plane(normal_vector=Vector(['1','1','1']), constant_term='1')
p2 = Plane(normal_vector=Vector(['0','1','1']), constant_term='2')
s = LinearSystem([p1,p2])
r = s.compute_rref()
if not (r[0] == Plane(normal_vector=Vector(['1','0','0']), constant_term='-1') and
        r[1] == p2):
    print 'test case 1 failed'

p1 = Plane(normal_vector=Vector(['1','1','1']), constant_term='1')
p2 = Plane(normal_vector=Vector(['1','1','1']), constant_term='2')
s = LinearSystem([p1,p2])
r = s.compute_rref()
if not (r[0] == p1 and
        r[1] == Plane(constant_term='1')):
    print 'test case 2 failed'

p1 = Plane(normal_vector=Vector(['1','1','1']), constant_term='1')
p2 = Plane(normal_vector=Vector(['0','1','0']), constant_term='2')
p3 = Plane(normal_vector=Vector(['1','1','-1']), constant_term='3')
p4 = Plane(normal_vector=Vector(['1','0','-2']), constant_term='2')
s = LinearSystem([p1,p2,p3,p4])
r = s.compute_rref()
if not (r[0] == Plane(normal_vector=Vector(['1','0','0']), constant_term='0') and
        r[1] == p2 and
        r[2] == Plane(normal_vector=Vector(['0','0','-2']), constant_term='2') and
        r[3] == Plane()):
    print 'test case 3 failed'

p1 = Plane(normal_vector=Vector(['0','1','1']), constant_term='1')
p2 = Plane(normal_vector=Vector(['1','-1','1']), constant_term='2')
p3 = Plane(normal_vector=Vector(['1','2','-5']), constant_term='3')
s = LinearSystem([p1,p2,p3])
r = s.compute_rref()
if not (r[0] == Plane(normal_vector=Vector(['1','0','0']), constant_term=Decimal('23')/Decimal('9')) and
        r[1] == Plane(normal_vector=Vector(['0','1','0']), constant_term=Decimal('7')/Decimal('9')) and
        r[2] == Plane(normal_vector=Vector(['0','0','1']), constant_term=Decimal('2')/Decimal('9'))):
    print 'test case 4 failed'

In [135]:
x=Vector([1,2])

In [16]:
p0.normal_vector.coordinates[0]*2.1

TypeError: unsupported operand type(s) for *: 'Decimal' and 'float'

In [44]:
print x

Vector: (Decimal('1'), Decimal('2'))


In [45]:
print Vector((8.218,-9.341))+Vector((-1.129,2.111))

Vector: (Decimal('7.089'), Decimal('-7.230'))


In [46]:
print Vector((7.119,8.215))-Vector((-8.223,0.878))

Vector: (Decimal('15.342'), Decimal('7.337'))


In [47]:
print Vector((1.671,-1.012,-0.318))*7.41

Vector: (Decimal('12.38211'), Decimal('-7.49892'), Decimal('-2.35638'))


In [123]:
print Vector((-0.221,7.437)).magnitude()

7.44028292472806439850041394956


In [136]:
print Vector((8.813,-1.331,-6.247)).magnitude()

10.8841875672922873421533791227


In [71]:
print Vector((5.581,-2.136)).direction()

Vector: (Decimal('0.933935214086640319028128029228'), Decimal('-0.357442325262329998466956006169'))


In [72]:
print Vector((1.996,3.108,-4.554)).direction()

Vector: (Decimal('0.340401295943301369107620613160'), Decimal('0.530043701298487302197637708267'), Decimal('-0.776647044952802823104260657481'))


In [139]:
angle(Vector((3.183,-7.627)),Vector((-2.668,5.319)))

3.0720263098372476

In [98]:
angle(Vector((7.35,0.221,5.188)),Vector((2.751,8.259,3.985)))*180/math.pi

60.27581120523091

In [99]:
dot(Vector((7.887,4.138)),Vector((-8.802,6.776)))

Decimal('-41.382286')

In [100]:
dot(Vector((-5.955,-4.904,-1.874)),Vector((-4.496,-8.755,7.103)))

Decimal('56.397178')

In [124]:
is_parallel(Vector((-7.579,-7.88)),Vector((22.737,23.64)))

True

In [125]:
is_ortogonal(Vector((-7.579,-7.88)),Vector((22.737,23.64)))

False

In [126]:
is_parallel(Vector((-2.029,9.97,4.172)),Vector((-9.231,-6.639,-7.245)))

False

In [127]:
is_ortogonal(Vector((-2.029,9.97,4.172)),Vector((-9.231,-6.639,-7.245)))

False

In [128]:
is_parallel(Vector((-2.328,-7.284,-1.214)),Vector((-1.821,1.072,-2.94)))

False

In [144]:
is_ortogonal(Vector((-2.328,-7.284,-1.214)),Vector((-1.821,1.072,-2.94)))

True

In [107]:
x=Vector((3.039,1.879)).projection(Vector((0.825,2.036)))

In [108]:
print x[0]

Vector: (Decimal('1.08260696248446669557997323205'), Decimal('2.67174275832530205115251575813'))


In [109]:
x=Vector((-9.88,-3.264,-8.159)).projection(Vector((-2.155,-9.353,-9.473)))

In [110]:
print x[1]

Vector: (Decimal('-8.35008104319576223434679936276'), Decimal('3.37606125428771963905075896060'), Decimal('-1.43374604278118591460196304568'))


In [111]:
x=Vector((3.009,-6.172,3.692,-2.51)).projection(Vector((6.404,-9.144,2.759,8.718)))

In [112]:
print x[0]

Vector: (Decimal('1.96851616721408987630529774899'), Decimal('-2.81076074843935631307552195764'), Decimal('0.848084963357850401112791456819'), Decimal('2.67981323325615795465796155148'))


In [113]:
print x[1]

Vector: (Decimal('1.04048383278591012369470225101'), Decimal('-3.36123925156064368692447804236'), Decimal('2.84391503664214959888720854318'), Decimal('-5.18981323325615795465796155148'))


In [114]:
print x[0]+x[1]

Vector: (Decimal('3.00900000000000000000000000000'), Decimal('-6.17200000000000000000000000000'), Decimal('3.69200000000000000000000000000'), Decimal('-2.51000000000000000000000000000'))


In [115]:
print cross_product(Vector([8.462,7.893,-8.187]),Vector([6.984,-5.975,4.778]))

Vector: (Decimal('-11.204571'), Decimal('-97.609444'), Decimal('-105.685162'))


In [116]:
cross_product(Vector([-8.987,-9.838,5.031]),Vector([-4.268,-1.861,-8.866])).magnitude()

Decimal('142.122221401846340546984269930')

In [118]:
cross_product(Vector([1.5,9.547,3.691]),Vector([-6.007,0.124,5.772])).magnitude()*Decimal('.5')

Decimal('42.5649373994189318366020004976')

In [2]:
x = Line(Vector([Decimal(4.046),Decimal(2.836)]),Decimal(1.21))
y = Line(Vector([Decimal(10.115),Decimal(7.09)]),Decimal(3.025))
intersection(x,y)

'equal'

In [3]:
x = Line(Vector([Decimal(7.204),Decimal(3.182)]),Decimal(8.68))
y = Line(Vector([Decimal(8.172),Decimal(4.114)]),Decimal(9.883))
intersection(x,y)

(Decimal('1.17277663546464155833736023125'),
 Decimal('0.0726955116633319428771277112347'))

In [4]:
x = Line(Vector([Decimal(1.182),Decimal(5.562)]),Decimal(6.744))
y = Line(Vector([Decimal(1.773),Decimal(8.343)]),Decimal(9.525))
intersection(x,y)

'parallel'

In [23]:
float(dot(Vector([Decimal(1.182),Decimal(5.562)]),Vector([Decimal(1.773),Decimal(8.343)])))

48.499452000000005

In [22]:
Vector([Decimal(1.182),Decimal(5.562)]).magnitude()*Vector([Decimal(1.773),Decimal(8.343)]).magnitude()

48.499452

In [21]:
Vector([Decimal(1.773),Decimal(8.343)]).magnitude()

8.529312867986494

In [83]:
x = Plane(Vector([-0.412,3.806,0.728]),-3.46)
y = Plane(Vector([1.03,-9.515,-1.82]),8.65)
intersection(x,y)

'equal'

In [27]:
x = Plane(Vector([2.611,5.528,0.283]),4.6)
y = Plane(Vector([7.715,8.306,5.342]),3.76)
intersection(x,y)

(Decimal('-0.831155934335784371114855862924'),
 Decimal('1.22470118389123238742936338242'))

In [28]:
x = Plane(Vector([-7.926,8.625,-7.212]),-7.952)
y = Plane(Vector([-2.642,2.875,-2.404]),-2.443)
intersection(x,y)

'parallel'

In [17]:
is_parallel(x.normal_vector,y.normal_vector)

True

In [19]:
print x.basepoint

Vector: (Decimal('8.39805825242718437978850294174'), Decimal('0'), Decimal('0'))


In [20]:
print y.basepoint

Vector: (Decimal('8.39805825242718481094307561170'), Decimal('0'), Decimal('0'))


In [21]:
x.basepoint == y.basepoint

True

In [24]:
x.normal_vector.direction()*-1 == y.normal_vector.direction()

True

In [309]:
p1 = Plane(normal_vector=Vector(['0.786','0.786','0.588']), constant_term='-0.714')
p2 = Plane(normal_vector=Vector(['-0.138','-0.138','0.244']), constant_term='0.319')
s = LinearSystem([p1,p2])
print s.compute_triangular_form().indices_of_first_nonzero_terms_in_each_row()

[0, 2]


In [318]:
p1 = Plane(normal_vector=Vector(['0.786','0.786','0.588']), constant_term='-0.714')
p2 = Plane(normal_vector=Vector(['-0.138','-0.138','0.244']), constant_term='0.319')
s = LinearSystem([p1,p2])
print s.compute_rref()

Linear System:
Equation 1: x_3 = 0.558
Equation 2: x_1 + x_2 = -1.326


In [315]:
p1 = Plane(normal_vector=Vector(['8.631','5.112','-1.816']), constant_term='-5.113')
p2 = Plane(normal_vector=Vector(['4.315','11.132','-5.27']), constant_term='-6.775')
p3 = Plane(normal_vector=Vector(['-2.158','3.01','-1.727']), constant_term='-0.831')
s = LinearSystem([p1,p2,p3])
print s.compute_rref()

Linear System:
Equation 1: x_2 - 0.509x_3 = -0.492
Equation 2: x_1 + 0.091x_3 = -0.301
Equation 3: 0 = 0


In [316]:
p1 = Plane(normal_vector=Vector(['0.935','1.76','-9.365']), constant_term='-9.955')
p2 = Plane(normal_vector=Vector(['0.187','0.352','-1.873']), constant_term='-1.991')
p3 = Plane(normal_vector=Vector(['0.374','0.704','-3.746']), constant_term='-3.982')
p4 = Plane(normal_vector=Vector(['-0.561','-1.056','5.619']), constant_term='5.973')
s = LinearSystem([p1,p2,p3,p4])
print s.compute_rref()

Linear System:
Equation 1: x_1 + 1.882x_2 - 10.016x_3 = -10.647
Equation 2: 0 = 0
Equation 3: 0 = 0
Equation 4: 0 = 0


In [321]:
dot(Vector(['0.786','0.786','0.588']),Vector([-1.346,0,0.585]))

Decimal('-0.714132')